In [ ]:
import polars as pl

In [ ]:
gateway = (
    pl.scan_csv('gateway_report.csv')
)

In [ ]:
granted = (
    gateway
    .group_by(by='Auth. Status')
    .agg(pl.col('Licensee').count())
)
granted.collect()

In [ ]:
state_collect = (
    gateway
    .select(
        pl.col('State(s) Located In').value_counts(sort=True)
    )
)
state_collect.collect()

In [ ]:
states = ['AZ', 'CO', 'NM', 'NV']
state_dfs = {}
total = 0
for state in states:
    metrics = (
        gateway  
        .with_columns(
            pl.col('Auth. Status').str.to_uppercase().fill_null('NA'),
            pl.col('State(s) Located In').str.to_uppercase().fill_null('NA')
        )
        .filter(
            pl.col('Auth. Status') != "DENIED", 
            pl.col('State(s) Located In').is_in(states),
            pl.col('State(s) Located In').str.contains(state)
        )
    ).collect()
    state_total = metrics.shape[0]
    print(f'{state}: {state_total}')
    total += state_total
    state_dfs[state] = metrics
print(f'{total = }')



In [ ]:
from xlsxwriter import Workbook
with Workbook("gateway_report_update.xlsx") as wb:  
    for state, df in state_dfs.items():
        df.write_excel(
            workbook=wb,
            worksheet=state,
            autofit=True,
            freeze_panes=((1,0,0,0)
        )
    )